In [1]:
from bs4 import BeautifulSoup 
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
import pandas as pd
import matplotlib.pyplot as plt
import os
import numpy as np
import time
import re
import warnings
import json 
warnings.filterwarnings('ignore')

# Set up Funds Input 

In [2]:
# Funds Url
urlETF = 'https://www.moneydj.com/funddj/yb/yp302000.djhtm?a=ET000001'
urlTech = 'https://www.moneydj.com/funddj/yb/yp302000.djhtm?a=ET001001'
urlChina = 'https://www.moneydj.com/funddj/yb/yp302000.djhtm?a=ET001006'
urlValue = 'https://www.moneydj.com/funddj/yb/yp302000.djhtm?a=ET001007'
urlOTC = 'https://www.moneydj.com/funddj/yb/yp302000.djhtm?a=ET001008'
urlCommon = 'https://www.moneydj.com/funddj/yb/YP302000.djhtm?a=ET001005'

categories = ["ETF", "Tech", "China", "Value", "OTC", "Common"]
index_lists = pd.read_csv(".\data\index_list.csv", encoding = 'cp950', header = None).values.tolist()
index_list = [ele[0] for ele in index_lists]
index_list

['基金名稱',
 '成立日期',
 '興櫃交易代碼',
 '基金規模',
 '基金公司',
 '基金類型',
 '基金經理人',
 '計價幣別',
 '成立時規模',
 '投資區域',
 '保管銀行',
 '基金評等',
 '主要投資區域',
 '單筆最低申購',
 '風險報酬等級',
 '定時定額',
 '管理年費(%)',
 '手續費(%)',
 '保管費(%)',
 '基金統編',
 '配息頻率',
 '費用備註',
 '文件下載',
 '投資標的']

In [3]:
def contents_output(df_contents, cate):
    output_path = str(os.path.join(os.getcwd(), 'data', cate))
    df_contents.to_csv(os.path.join(output_path, "{}_basic_info.csv".format(cate)), encoding = 'cp950',  header = 0)

In [4]:
def scratch_basicInfo(url):
    main_driver.get(url)
    main_driver.implicitly_wait(10)
    funds_basic_info = main_driver.find_element_by_xpath("//*[@id='article']/form/div[1]/div/ul/li[1]/a")
    funds_basic_info.click()
    main_driver.implicitly_wait(10)
    simpleTable =  main_driver.find_element_by_xpath("//*[@id='article']/form/table[1]/tbody" )
    rows = simpleTable.find_elements_by_tag_name("tr")
    contents = []
    
    for row in rows[:-2]:
        cols = row.find_elements_by_tag_name("td")
        line = []
        for i, col in enumerate(cols):
            if i == 1 or i == 3: #only include the value
                line.append(col.text)
            elif i == 2: # for two columns
                contents.append(line)
                line = []
        contents.append(line)
    return contents

In [5]:
results_path = str(os.path.join(os.getcwd(), 'data'))
chrome_options = webdriver.ChromeOptions() 
prefs = {'download.default_directory' : results_path}
chrome_options.add_experimental_option('prefs', prefs)

main_driver = webdriver.Chrome('C:\\webdrivers\\chromedriver', options = chrome_options)  
main_driver.get(urlCommon)
main_driver.implicitly_wait(5)

In [6]:
fundTable = main_driver.find_element_by_xpath('//*[@id="oMainTable"]/tbody')
rows = fundTable.find_elements_by_tag_name("tr")
print("總共幾個Funds: %i" %len(rows))
print("---------------------------------")

links = []
for row in rows:
    firstCol = row.find_elements_by_tag_name("td")[3]
    fundName = firstCol.text
    print("現在正在處理: %s" %fundName)
    print(firstCol.find_elements_by_tag_name('a')[0].get_attribute('href'))
    links.append(firstCol.find_elements_by_tag_name('a')[0].get_attribute('href'))

總共幾個Funds: 91
---------------------------------
現在正在處理: 匯豐成功基金
https://www.moneydj.com/funddj/ya/yp010000.djhtm?a=AC0005
現在正在處理: 富蘭克林華美傳產基金
https://www.moneydj.com/funddj/ya/yp010000.djhtm?a=ACFT13
現在正在處理: 兆豐國際豐台灣基金
https://www.moneydj.com/funddj/ya/yp010000.djhtm?a=ACII21
現在正在處理: 保德信新世紀基金
https://www.moneydj.com/funddj/ya/yp010000.djhtm?a=ACML12
現在正在處理: 貝萊德寶利基金
https://www.moneydj.com/funddj/ya/yp010000.djhtm?a=ACBR01
現在正在處理: 安聯台灣智慧基金
https://www.moneydj.com/funddj/ya/yp010000.djhtm?a=ACDD19
現在正在處理: 野村積極成長基金
https://www.moneydj.com/funddj/ya/yp010000.djhtm?a=ACKH07
現在正在處理: 安聯台灣大壩基金
https://www.moneydj.com/funddj/ya/yp010000.djhtm?a=ACDD01
現在正在處理: 野村成長基金
https://www.moneydj.com/funddj/ya/yp010000.djhtm?a=ACIC04
現在正在處理: 路博邁AR台灣股票基金T月配型(基金之配息來源可能為本金)
https://www.moneydj.com/funddj/ya/yp010000.djhtm?a=ACNB02
現在正在處理: 路博邁AR台灣股票基金T累積型(基金之配息來源可能為本金)
https://www.moneydj.com/funddj/ya/yp010000.djhtm?a=ACNB01
現在正在處理: 新光台灣富貴基金
https://www.moneydj.com/funddj/ya/yp010000.djhtm?a=AC0021
現在正在處理: 柏瑞巨人

In [7]:
concated_df = []
for link in links:
    contents = scratch_basicInfo(link)
    print("現在正在處理: %s" %contents[0][0])
    df = pd.DataFrame(contents, index = index_list)
    concated_df.append(df)

result = pd.concat(concated_df, axis=1)

現在正在處理: 匯豐成功基金
現在正在處理: 富蘭克林華美傳產基金
現在正在處理: 兆豐國際豐台灣基金
現在正在處理: 保德信新世紀基金
現在正在處理: 貝萊德寶利基金
現在正在處理: 安聯台灣智慧基金
現在正在處理: 野村積極成長基金
現在正在處理: 安聯台灣大壩基金
現在正在處理: 野村成長基金
現在正在處理: 路博邁AR台灣股票基金T月配型(基金之配息來源可能為本金)
現在正在處理: 路博邁AR台灣股票基金T累積型(基金之配息來源可能為本金)
現在正在處理: 新光台灣富貴基金
現在正在處理: 柏瑞巨人基金
現在正在處理: 匯豐龍鳳基金
現在正在處理: 匯豐龍鳳基金I類型
現在正在處理: 匯豐台灣精典基金
現在正在處理: 凱基開創基金
現在正在處理: 保德信金滿意基金
現在正在處理: 凱基台灣精五門基金
現在正在處理: 華南永昌永昌基金
現在正在處理: 保德信高成長基金
現在正在處理: 德信大發基金
現在正在處理: 兆豐國際第一基金
現在正在處理: 兆豐國際國民基金
現在正在處理: 合庫台灣基金
現在正在處理: 野村優質基金
現在正在處理: 日盛MIT主流基金
現在正在處理: 摩根台灣增長基金
現在正在處理: 華南永昌物聯網精選基金
現在正在處理: 群益奧斯卡基金
現在正在處理: 統一大滿貫基金
現在正在處理: 聯博大利基金
現在正在處理: 台新主流基金
現在正在處理: 復華高成長基金
現在正在處理: 富邦富邦基金
現在正在處理: 保德信第一基金
現在正在處理: 復華復華基金
現在正在處理: 群益長安基金
現在正在處理: 日盛上選基金
現在正在處理: 日盛首選基金
現在正在處理: 日盛精選五虎基金
現在正在處理: 摩根台灣金磚基金-累積型
現在正在處理: 德銀遠東DWS台灣旗艦基金
現在正在處理: 宏利台灣動力基金A類
現在正在處理: 國泰小龍基金(台幣)
現在正在處理: 永豐永豐基金
現在正在處理: 野村台灣運籌基金
現在正在處理: 野村鴻運基金
現在正在處理: 景順潛力基金
現在正在處理: 富邦台灣心基金
現在正在處理: 群益馬拉松基金
現在正在處理: 國泰國泰基金
現在正在處理: 國泰國泰基金-I類型
現在正在處理: 景順主流基金
現在正在處理: 群益葛萊美基金
現在正在處理: 瀚亞菁華基金
現在正在處理: 富蘭克林華美第一富基金
現在正在處理: 復華全方

In [8]:
result.head(5)

,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
基金名稱,匯豐成功基金,富蘭克林華美傳產基金,兆豐國際豐台灣基金,保德信新世紀基金,貝萊德寶利基金,安聯台灣智慧基金,野村積極成長基金,安聯台灣大壩基金,野村成長基金,路博邁AR台灣股票基金T月配型(基金之配息來源可能為本金),...,摩根台灣金磚基金-機構法人型,群益馬拉松基金I類型,路博邁AR台灣股票基金N月配型(基金之配息來源可能為本金),路博邁AR台灣股票基金N累積型(基金之配息來源可能為本金),宏利臺灣股息收益基金I類,宏利台灣動力基金I類,國泰小龍基金-I類型(美元),永豐永豐基金-I類型,路博邁AR台灣股票基金I月配型(基金之配息來源可能為本金),路博邁AR台灣股票基金I累積型(基金之配息來源可能為本金)
成立日期,1990/08/13,2011/05/16,2008/08/22,2000/03/04,1999/05/10,2008/04/24,1994/01/12,2000/04/10,1999/11/16,2017/09/13,...,2018/07/17,2018/06/15,2019/04/01,2019/04/01,2019/05/20,2019/05/20,2019/02/11,,,
興櫃交易代碼,,,T0110Y.TE,,,,,,,,...,,,,,,,,,,
基金規模,6.49億元(台幣) 規模日期:2019/04/30,3.44億元(台幣) 規模日期:2019/04/30,4.62億元(台幣) 規模日期:2019/04/30,6.36億元(台幣) 規模日期:2019/04/30,5.17億元(台幣) 規模日期:2019/04/30,4.91億元(台幣) 規模日期:2019/04/30,4.12億元(台幣) 規模日期:2019/04/30,44.21億元(台幣) 規模日期:2019/04/30,5.85億元(台幣) 規模日期:2019/04/30,2.65億元(台幣) 規模日期:2019/04/30,...,0千元(台幣) 規模日期:2019/04/30,0千元(台幣) 規模日期:2019/04/30,0千元(台幣) 規模日期:2019/04/30,0千元(台幣) 規模日期:2019/04/30,,,0千元(台幣) 規模日期:2019/04/30,,,
基金公司,匯豐中華投信,富蘭克林華美投信,兆豐國際投信,保德信投信,貝萊德投信,安聯投信,野村投信,安聯投信,野村投信,路博邁投信,...,摩根投信,群益投信,路博邁投信,路博邁投信,宏利投信,宏利投信,國泰投信,永豐投信,路博邁投信,路博邁投信


In [9]:
contents_output(result, cate = 'Common')